<a href="https://colab.research.google.com/github/NamanGupta-0509/Mathematical-Trading-Strategies/blob/main/Assignment1/Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import yfinance as yf
import numpy as np
import pandas as pd
import math

In [39]:
securities = ["AAPL", "AMZN", "TSLA", "MSFT", "META", "^GSPC", "^DJI", "^IXIC", "^N225", "^FTSE"]

df = []
for i in range(10):
    df.append(yf.download(securities[i], start='2010-01-01', end='2023-05-01'))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [40]:
################# DAILY RETURNS ####################

for i in range(10):
    df[i]['Daily Returns'] = 100*(df[i]['Close'] - df[i]['Open'])/(df[i]['Open'])
df[0]

,Open,High,Low,Close,Adj Close,Volume,Daily Returns
Date,,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.496294,493729600,0.271752
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.507526,601904800,-0.102519
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.404015,552160000,-1.590632
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.392177,477131200,-0.552545
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.434672,447610800,0.798858
...,...,...,...,...,...,...,...
2023-04-24,165.000000,165.600006,163.889999,165.330002,165.101639,41949600,0.200001
2023-04-25,165.190002,166.309998,163.729996,163.770004,163.543793,48714100,-0.859615
2023-04-26,163.059998,165.279999,162.800003,163.759995,163.533798,45498800,0.429288


In [41]:
################## CUMMULATIVE RETURNS #################
df_cr = []
for i in range(10):
    df_cr.append(100*(df[i]['Close'][-1] - df[i]['Open'][0])/df[i]['Open'][0])
df_cr

[2126.0412456481768,
 1447.889863460436,
 12871.838412855475,
 903.4617940700147,
 471.51013480260417,
 273.4219174667591,
 226.90223515023416,
 432.8856022455222,
 171.99090497722563,
 45.404501092812225]

In [42]:
####################### VOLATILITY ######################

df_volatility = []
for i in range(10):
    days = df[i].shape[0]
    vol = []
    for j in range(1, days):
        vol.append(100*np.log(df[i]['Close'][j] / df[i]['Close'][j-1]))
    df_volatility.append(np.std(vol))
print(df_volatility)  # Daily Volatility(in %)

df_annvol = [math.sqrt(252)*i for i in df_volatility]
print(df_annvol)  # Annual Volatility(in %)

[1.8033946761804027, 2.0853414204603156, 3.6016910810695313, 1.6480958303226068, 2.574164692547689, 1.1223192490289855, 1.0860515571699811, 1.301638303251528, 1.314440840355142, 1.026578067498893]
[28.628002973227222, 33.103768783201055, 57.17507339873613, 26.16271022301673, 40.86359766122536, 17.81626574730879, 17.240533987597306, 20.662867484157715, 20.866101460118756, 16.2964216079721]


In [43]:
################### SHARPE RATIO ###################
# Taking risk free rate = 5.52%

df_sr = []
for i in range(10):
    ret = (1+(df_cr[i]/100)**(365/4898)-1)*100
    df_sr.append((ret-5.52)/df_annvol[i])
df_sr


[4.193901351150468,
 3.519807467272784,
 2.4153647676631804,
 4.29253033936215,
 2.611874925862508,
 5.73990407423054,
 5.845298264198256,
 5.130844059792844,
 4.725548680748389,
 5.446959013624661]

In [44]:
############## SORTIONO RATIO #################

df_sor = []

for i in range(10):
    # Calculate downside deviation
    square_sum = 0
    n = 0
    for j in df[i]['Daily Returns']:
        if j < 0:
            square_sum += j*j
        n += 1
    dd = (square_sum / n)**0.5
    df_sor.append((ret-5.52)/dd)
df_sor

[90.65885976443391,
 77.46257347567251,
 43.194074938698,
 101.08022859969445,
 68.67436832113067,
 131.36929371667168,
 135.88428995582882,
 121.38225352899386,
 130.66395567090004,
 119.34775065243302]

In [45]:
############### MAXIMUM DRAWDOWN #################

df_mdd = []

for i in range(10):
    data = list(df[i]['Close'])
    days = len(data)
    peak = data[0]
    mdd = []
    prices_between = []
    for i in range(1, days):
        if data[i] >= peak:
            if prices_between:
                mdd.append(100*(peak - min(prices_between))/peak)
            peak = data[i]
            prices_between = []
        else:
            prices_between.append(data[i])
    df_mdd.append(-1*max(mdd))

df_mdd

[-44.37687017144154,
 -34.10378304630068,
 -60.626538538206894,
 -28.23529392738191,
 -53.62280995413482,
 -33.92496000265327,
 -37.086171369594325,
 -30.115673337317634,
 -31.79889650850191,
 -36.60552329617582]

In [46]:
cols = ['Metrics', 'AAPL', 'AMZN', 'TSLA', 'MSFT', 'META']
stock_result = pd.DataFrame(columns=cols)
stock_result

,Metrics,AAPL,AMZN,TSLA,MSFT,META


In [47]:
row = ['Cummulative Returns'] + df_cr[:5]
row = pd.Series(row, index=stock_result.columns)
stock_result = stock_result.append(row, ignore_index=True)

row = ['Volatility'] + df_annvol[:5]
row = pd.Series(row, index=stock_result.columns)
stock_result = stock_result.append(row, ignore_index=True)

row = ['Sharpe Ratio'] + df_sr[:5]
row = pd.Series(row, index=stock_result.columns)
stock_result = stock_result.append(row, ignore_index=True)

row = ['Sortiono Ratio'] + df_sor[:5]
row = pd.Series(row, index=stock_result.columns)
stock_result = stock_result.append(row, ignore_index=True)

row = ['Max Drawdown'] + df_mdd[:5]
row = pd.Series(row, index=stock_result.columns)
stock_result = stock_result.append(row, ignore_index=True)



<ipython-input-47-fe738362a202>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_result = stock_result.append(row, ignore_index=True)
<ipython-input-47-fe738362a202>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_result = stock_result.append(row, ignore_index=True)
<ipython-input-47-fe738362a202>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_result = stock_result.append(row, ignore_index=True)
<ipython-input-47-fe738362a202>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_result = stock_result.append(row, ignore_index=True)
<ipython-input-47-fe738362a202>:19: FutureWarning: The frame.append method is depr

In [48]:
stock_result

,Metrics,AAPL,AMZN,TSLA,MSFT,META
0,Cummulative Returns,2126.041246,1447.889863,12871.838413,903.461794,471.510135
1,Volatility,28.628003,33.103769,57.175073,26.162710,40.863598
2,Sharpe Ratio,4.193901,3.519807,2.415365,4.292530,2.611875
3,Sortiono Ratio,90.658860,77.462573,43.194075,101.080229,68.674368
4,Max Drawdown,-44.376870,-34.103783,-60.626539,-28.235294,-53.622810


In [49]:
cols = ['Metrics', "GSPC", "DJI", "IXIC", "N225", "FTSE"]
index_result = pd.DataFrame(columns=cols)

In [50]:
row = ['Cummulative Returns'] + df_cr[5:]
row = pd.Series(row, index=index_result.columns)
index_result = index_result.append(row, ignore_index=True)

row = ['Volatility'] + df_annvol[:5]
row = pd.Series(row, index=index_result.columns)
index_result = index_result.append(row, ignore_index=True)

row = ['Sharpe Ratio'] + df_sr[:5]
row = pd.Series(row, index=index_result.columns)
index_result = index_result.append(row, ignore_index=True)

row = ['Sortiono Ratio'] + df_sor[:5]
row = pd.Series(row, index=index_result.columns)
index_result = index_result.append(row, ignore_index=True)

row = ['Max Drawdown'] + df_mdd[:5]
row = pd.Series(row, index=index_result.columns)
index_result = index_result.append(row, ignore_index=True)

<ipython-input-50-b76fee825a65>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  index_result = index_result.append(row, ignore_index=True)
<ipython-input-50-b76fee825a65>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  index_result = index_result.append(row, ignore_index=True)
<ipython-input-50-b76fee825a65>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  index_result = index_result.append(row, ignore_index=True)
<ipython-input-50-b76fee825a65>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  index_result = index_result.append(row, ignore_index=True)
<ipython-input-50-b76fee825a65>:19: FutureWarning: The frame.append method is depr

In [51]:
index_result

,Metrics,GSPC,DJI,IXIC,N225,FTSE
0,Cummulative Returns,273.421917,226.902235,432.885602,171.990905,45.404501
1,Volatility,28.628003,33.103769,57.175073,26.162710,40.863598
2,Sharpe Ratio,4.193901,3.519807,2.415365,4.292530,2.611875
3,Sortiono Ratio,90.658860,77.462573,43.194075,101.080229,68.674368
4,Max Drawdown,-44.376870,-34.103783,-60.626539,-28.235294,-53.622810
